# Setting

In [1]:
from urllib.parse import urlparse, urlunparse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException 
import time, logging

In [2]:
# 기본 로거 생성
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(message)s')
logger = logging.getLogger()
logger.info('크롤링 시작')

2024-03-18 19:19:37,907 | 크롤링 시작


In [3]:
# 수집할 리뷰 개수
n = 100

# 수집하고자 하는 상품의 URL
# raw_url = 'https://smartstore.naver.com/coffeemall/products/4442922120?NaPm=ct%3Dltwgl1iw%7Cci%3D7a1714772f09653c3e3214b3d5832642bd0db583%7Ctr%3Dslsl%7Csn%3D164274%7Chk%3D6932c84a73a753e12b841eb03f4d91e64e983955'
raw_url = 'https://smartstore.naver.com/motherfatherlab/products/8325101187?NaPm=ct%3Dltwi1xug%7Cci%3D118679c7098df43fa89bb9fd7361a95306b3328b%7Ctr%3Dslsl%7Csn%3D4740545%7Chk%3Da0e59755c2fd2edee1441d9f8f25a1b3c0e71552'

# URL 분해 -> schem, netloc, path, parms, query, fragement
parsed_url = urlparse(raw_url)

# parms, query, fragement 3가지 값을 공백으로 설정하여 URL 재구성
url = urlunparse((parsed_url.scheme, parsed_url.netloc, parsed_url.path + '/', '', '', ''))

logger.info(f'변경된 URL : {url}')

2024-03-18 19:19:37,914 | 변경된 URL : https://smartstore.naver.com/motherfatherlab/products/8325101187/


In [4]:
# 크롬 드라이버 설치
driver_path = ChromeDriverManager().install()
logger.info(f'크롬 드라이버 설치 경로 : {driver_path}')

# 브라우저 인스턴스 생성, url 오픈
driver = webdriver.Chrome(service=Service(driver_path))
driver.get(url)
driver.implicitly_wait(10)

2024-03-18 19:19:37,919 | ====== WebDriver manager ======
2024-03-18 19:19:38,389 | Get LATEST chromedriver version for google-chrome
2024-03-18 19:19:38,640 | Get LATEST chromedriver version for google-chrome
2024-03-18 19:19:38,901 | Driver [C:\Users\njh26\.wdm\drivers\chromedriver\win64\122.0.6261.128\chromedriver-win32/chromedriver.exe] found in cache
2024-03-18 19:19:38,902 | 크롬 드라이버 설치 경로 : C:\Users\njh26\.wdm\drivers\chromedriver\win64\122.0.6261.128\chromedriver-win32/chromedriver.exe


# Crawling

In [5]:
# 상품명 가져오기
product_name = driver.find_element(By.CLASS_NAME, '_22kNQuEXmb._copyable').text
product_name = product_name.replace(':', '')    # * 가 있는 경우 to_csv에서 오류 발생
### df.to_csv에서 오류가 발생한 경우 위 코드에서 문자를 추가로 변환해야 함
logger.info(f'상품명 : {product_name}')

2024-03-18 19:19:40,809 | 상품명 : 모부당 떡볶이에빠진 치즈가래 떡볶이맛 떡 답례 영양 개업 맛집 아침대용 맛있는 칠순 이사 냉동 [원산지상세설명에 표시]


In [6]:
# 리뷰 탭으로 이동
review_tab = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[3]/div[3]/ul/li[2]/a')
review_tab.click()

# 최신순 정렬로 변경
# 리뷰 탭으로 이동해야만 최신순 버튼을 찾을 수 있음
latest = driver.find_element(By.XPATH, '//*[@id="REVIEW"]/div/div[3]/div[1]/div[1]/ul/li[2]/a')
latest.click()
time.sleep(0.1)

In [7]:
# 리뷰를 저장할 리스트 생성
reviews = []
ratings = []
days = []
options = []
buyer_categorys = []
first_keywords = []
second_keywords = []
third_keywords = []

# 크롤링 함수
def crawling():

    # 리뷰 박스
    review_boxs = driver.find_elements(By.CSS_SELECTOR, 'ul._2ms2i3dD92 > li')

    # 각 리뷰 박스별로 데이터 수집
    for (j, review_box) in enumerate(review_boxs):

        # 리뷰
        review = review_box.find_element(By.CSS_SELECTOR,'div._1kMfD5ErZ6 > span._2L3vDiadT9')
        reviews.append(review.text)

        # 별점
        rating = review_box.find_element(By.CSS_SELECTOR,'div._3HKlxxt8Ii > div._2V6vMO_iLm > em._15NU42F3kT')
        ratings.append(rating.text)
        
        # 리뷰작성일
        day = review_box.find_element(By.CSS_SELECTOR,'div.iWGqB6S4Lq > span._2L3vDiadT9')
        days.append('20' + day.text)  # 24.03.07 형식으로 수집되어 앞에 20 추가
        
        # 구매옵션
        # 키워드와 같이 있어 수집 이후에 \n 으로 슬라이싱
        option_and_keyword = review_box.find_element(By.CSS_SELECTOR,'div._3HKlxxt8Ii > div._2FXNMst_ak')
        options.append(option_and_keyword.text.split('\n')[0])

        # 구매자 정보
        try : 
            buyer_category = review_box.find_element(By.CSS_SELECTOR,'dl.XbGQRlzveO > div._3F8sJXhFeW')       
            buyer_categorys.append(buyer_category.text)
        except NoSuchElementException : 
            buyer_categorys.append('')  # 구매자 정보가 없는 경우 공백을 입력

        # 키워드
        keyword_all = review_box.find_elements(By.CSS_SELECTOR,'dl.XbGQRlzveO > div._1QLwBCINAr > dd')
        first_keywords.append(keyword_all[0].text)
        second_keywords.append(keyword_all[1].text)
        third_keywords.append(keyword_all[2].text)
        
        logger.info(f'\t {j+1}번째 리뷰 획득')

In [8]:
# 페이지 수
pages = int((n-1) / 20 + 1)   #페이지당 리뷰 20개

# 다음 페이지 버튼
next = driver.find_element(By.CLASS_NAME, 'fAUKm1ewwo._2Ar8-aEUTq')

# 크롤링
for i in range(1, pages + 1):
    if i == 1:
        crawling()
        logger.info(f'--{i} 페이지 리뷰 수집 완료--')
    else:
        # 다음 페이지로 이동
        next.click()
        time.sleep(0.5)
        crawling()
        logger.info(f'--{i} 페이지 리뷰 수집 완료--')

2024-03-18 19:19:42,040 | 	 1번째 리뷰 획득
2024-03-18 19:19:52,129 | 	 2번째 리뷰 획득
2024-03-18 19:19:52,223 | 	 3번째 리뷰 획득
2024-03-18 19:19:52,308 | 	 4번째 리뷰 획득
2024-03-18 19:20:02,361 | 	 5번째 리뷰 획득
2024-03-18 19:20:02,422 | 	 6번째 리뷰 획득
2024-03-18 19:20:12,488 | 	 7번째 리뷰 획득
2024-03-18 19:20:22,548 | 	 8번째 리뷰 획득
2024-03-18 19:20:32,607 | 	 9번째 리뷰 획득
2024-03-18 19:20:32,668 | 	 10번째 리뷰 획득
2024-03-18 19:20:32,728 | 	 11번째 리뷰 획득
2024-03-18 19:20:42,801 | 	 12번째 리뷰 획득
2024-03-18 19:20:52,868 | 	 13번째 리뷰 획득
2024-03-18 19:21:02,948 | 	 14번째 리뷰 획득
2024-03-18 19:21:13,017 | 	 15번째 리뷰 획득
2024-03-18 19:21:23,093 | 	 16번째 리뷰 획득
2024-03-18 19:21:33,164 | 	 17번째 리뷰 획득
2024-03-18 19:21:33,225 | 	 18번째 리뷰 획득
2024-03-18 19:21:43,288 | 	 19번째 리뷰 획득
2024-03-18 19:21:53,354 | 	 20번째 리뷰 획득
2024-03-18 19:21:53,355 | --1 페이지 리뷰 수집 완료--
2024-03-18 19:22:03,977 | 	 1번째 리뷰 획득
2024-03-18 19:22:04,039 | 	 2번째 리뷰 획득
2024-03-18 19:22:14,098 | 	 3번째 리뷰 획득
2024-03-18 19:22:14,160 | 	 4번째 리뷰 획득
2024-03-18 19:22:14,221 | 	 5번째 

In [9]:
# 키워드명 가져오기
keyword_name = driver.find_elements(By.CSS_SELECTOR,'dl.XbGQRlzveO > div._1QLwBCINAr > dt') 
keyword_name_list = [a.text for a in keyword_name][:3]

# 드라이버 종료
driver.quit()

In [10]:
# 크롤링 결과 저장
df = pd.DataFrame({
                'reviews': reviews, 
                'ratings': ratings, 
                'days': days,
                'options': options, 
                'buyer_categorys' : buyer_categorys,
                keyword_name_list[0] : first_keywords, 
                keyword_name_list[1] : second_keywords, 
                keyword_name_list[2] : third_keywords
                })

print(f'\n수집한 데이터는 다음과 같습니다. \n {df.head(4)}')

df.to_csv(f"{product_name}_{n}.csv", encoding='utf-8-sig')
logger.info('크롤링 완료')

2024-03-18 19:31:33,608 | 크롤링 완료



수집한 데이터는 다음과 같습니다. 
                                              reviews ratings         days  \
0                    독특하지만 매우 맛있지는 않다\n한번정도는 먹어볼만한 맛       5  2024.03.18.   
1                                        간식으로 구매~~~~       4  2024.03.18.   
2                              맛있네요 굿굿 배고플때 먹으면 딱이에요       5  2024.03.18.   
3  생각보다 매워요 제가 맵찔이 이지만 \n떡볶이를 먹는다면 매운떡볶이 맛 깔끔한 고추...       4  2024.03.17.   

                     options  buyer_categorys    유통기한       포장 맛 만족도  
0  수량: 1Box(12개입) / 맛선택: 치즈맛     구매자거주인원 2인싱글    평범해요  아주 넉넉해요  꼼꼼해요  
1  수량: 1Box(12개입) / 맛선택: 치즈맛                     평범해요   꽤 남았어요  적당해요  
2  수량: 1Box(12개입) / 맛선택: 치즈맛  구매자거주인원 2인신혼/부부  꽤 남았어요     꼼꼼해요  맛있어요  
3  수량: 1Box(12개입) / 맛선택: 치즈맛     구매자거주인원 1인싱글  꽤 남았어요     꼼꼼해요  맛있어요  
